# fastxtend
> fastxtend (fastai extensions) is a collection of useful tools and extensions I have created or adapted for version 2 of the fastai framework.

## Install

To install, run:
```
pip install git+https://github.com/warner-benjamin/fastxtend.git
```

Or to create an editable install:
```
git clone https://github.com/warner-benjamin/fastxtend.git
cd fastxtend
pip install -e ".[dev]"
```

## Requirements

fastxtend requires fastai to be installed. See http://docs.fast.ai/ for installation instructions.

## Documentation
https://warner-benjamin.github.io/fastxtend/